# Lesson 5: Reranking Retrieved Documents

**Objective**: Enhance the relevance of retrieved documents to improve the quality of generated responses.

**Topics**:
- Reranking techniques: monoT5, monoBERT, RankLLaMA, TILDEv2, Cohere ReRanker
- Trade-offs between speed and accuracy

**Practical Task**: Implement a reranking model and evaluate its impact on retrieval performance.

**Resources**:
- Cohere reranker
- Open-source alternative


In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "../data/Regulaciones cacao y chocolate 2003.pdf"
)
loader = PyPDFLoader(file_path)
splitted_doc = loader.load_and_split()

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

f:\repositorios\rag-course-fix\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
from langchain_qdrant import RetrievalMode
from langchain_qdrant import QdrantVectorStore

qdrant = QdrantVectorStore.from_documents(
    splitted_doc,
    embedding=embedding_model,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

query = "What is chocolate"
found_docs = qdrant.similarity_search(query)

In [7]:
found_docs

[Document(metadata={'source': '../data/Regulaciones cacao y chocolate 2003.pdf', 'page': 9, '_id': '79473b9272bd45f18a51a4899d5d04f1', '_collection_name': 'my_documents'}, page_content='Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nColumn 1 Column 2\nReserved Descriptions Designated Products\n—not less than 25 per cent total fat (cocoa\nbutter and milk fat).\n6.White chocolate The product obtained from cocoa butter, milk\nor milk products and sugars which contains not\nless than 20 per cent cocoa butter and not less\nthan 14 per cent dry milk solids obtained by\npartly or wholly dehydrating whole milk, semi-\nskimmed or skimmed milk, cream, or from\npartly or wholly dehydrated cream, butter or\nmilk fat, of which not less than 3.5 per cent is\nmilk fat.\n7.Filled chocolate or\nChocolate with … filling or\nChocolate with … centreThe filled product, the outer part of which\nconsists of a product specified in column\n2 of item 3, 4, 5 

In [17]:
retriever = qdrant.as_retriever(search_type="mmr", search_kwargs={"k": 10})
retrieved_docs = retriever.invoke("What is chocolate")

In [14]:
from rerankers import Reranker
from dotenv import load_dotenv
import os

load_dotenv()
# API (Cohere)
ranker = Reranker("cohere", lang='en', api_key=os.getenv("COHERE_API_KEY"))

Auto-updated model_name to rerank-english-v3.0 for API provider cohere
Loading APIRanker model rerank-english-v3.0


In [18]:
retrieved_docs

[Document(metadata={'source': '../data/Regulaciones cacao y chocolate 2003.pdf', 'page': 9, '_id': '79473b9272bd45f18a51a4899d5d04f1', '_collection_name': 'my_documents'}, page_content='Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nColumn 1 Column 2\nReserved Descriptions Designated Products\n—not less than 25 per cent total fat (cocoa\nbutter and milk fat).\n6.White chocolate The product obtained from cocoa butter, milk\nor milk products and sugars which contains not\nless than 20 per cent cocoa butter and not less\nthan 14 per cent dry milk solids obtained by\npartly or wholly dehydrating whole milk, semi-\nskimmed or skimmed milk, cream, or from\npartly or wholly dehydrated cream, butter or\nmilk fat, of which not less than 3.5 per cent is\nmilk fat.\n7.Filled chocolate or\nChocolate with … filling or\nChocolate with … centreThe filled product, the outer part of which\nconsists of a product specified in column\n2 of item 3, 4, 5 

In [21]:
str_docs = []

for doc in retrieved_docs:
    str_docs.append(doc.page_content)

In [22]:
str_docs

['Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nColumn 1 Column 2\nReserved Descriptions Designated Products\n—not less than 25 per cent total fat (cocoa\nbutter and milk fat).\n6.White chocolate The product obtained from cocoa butter, milk\nor milk products and sugars which contains not\nless than 20 per cent cocoa butter and not less\nthan 14 per cent dry milk solids obtained by\npartly or wholly dehydrating whole milk, semi-\nskimmed or skimmed milk, cream, or from\npartly or wholly dehydrated cream, butter or\nmilk fat, of which not less than 3.5 per cent is\nmilk fat.\n7.Filled chocolate or\nChocolate with … filling or\nChocolate with … centreThe filled product, the outer part of which\nconsists of a product specified in column\n2 of item 3, 4, 5 or 6 of this Schedule and\nconstitutes not less than 25 per cent of the total\nweight of the product, but does not include any\nfilled product, the inside of which consists of\nbakery 

In [ ]:
ranker.rank("What is chocolate", str_docs)

In [23]:
ranker.rank("What is chocolate", str_docs)

{'id': 'c567e03d-8e21-441e-b330-31050e8ef03a', 'results': [{'document': {'text': 'Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nSCHEDULE 1 Regulations 2, 3 and 6\nCOCOA AND CHOCOLATE PRODUCTS AND THEIR RESERVED DESCRIPTIONS\nColumn 1 Column 2\nReserved Descriptions Designated Products\n1.Cocoa butter The fat obtained from cocoa beans or parts of\ncocoa beans with the following characteristics:\n—\n—not more than 1.75 per cent free fatty acid\ncontent (expressed as oleic acid); and\n—for press cocoa butter, not more than\n0.35 per cent unsaponifiable matter\n(determined using petroleum ether); or\n—for other cocoa butter, not more than\n0.5 per cent unsaponifiable matter (so\ndetermined).\n(a) (a)  Cocoa powder or Cocoa The product obtained by converting into\npowder cocoa beans which have been cleaned,\nshelled and roasted, and which contains not\nless than 20 per cent cocoa butter, calculated\naccording to the weight of the dry mat

RankedResults(results=[Result(document=Document(text='Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nSCHEDULE 1 Regulations 2, 3 and 6\nCOCOA AND CHOCOLATE PRODUCTS AND THEIR RESERVED DESCRIPTIONS\nColumn 1 Column 2\nReserved Descriptions Designated Products\n1.Cocoa butter The fat obtained from cocoa beans or parts of\ncocoa beans with the following characteristics:\n—\n—not more than 1.75 per cent free fatty acid\ncontent (expressed as oleic acid); and\n—for press cocoa butter, not more than\n0.35 per cent unsaponifiable matter\n(determined using petroleum ether); or\n—for other cocoa butter, not more than\n0.5 per cent unsaponifiable matter (so\ndetermined).\n(a) (a)  Cocoa powder or Cocoa The product obtained by converting into\npowder cocoa beans which have been cleaned,\nshelled and roasted, and which contains not\nless than 20 per cent cocoa butter, calculated\naccording to the weight of the dry matter, and\nnot more than 9 pe